In [ ]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi

In [ ]:
symbols = ['A','AA']

In [ ]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2023, 1, 1))
df = df.collect()

In [ ]:
df

In [ ]:
ret1 = pi.simple_moving_average(df, 5)
ret2 = pi.simple_moving_average(ret1.df, 4)
ret2.df

In [ ]:
ret3 = pi.crossover(ret2.df, ret1.column, ret2.column, direction='up')
ret3.df

In [ ]:
ret3.df.filter(pl.col(ret3.column) == True)

In [ ]:
ret3.df

In [ ]:
index_name = 'ind'
column_name = 'test'
first_symbol = df[pi.SYMBOL_COLUMN].to_list()[0]
newdf = df.with_row_count(name=index_name).with_columns( 
                pl.when(
                    pl.col(pi.SYMBOL_COLUMN) == first_symbol) \
                    .then(pl.col(index_name) + 1) \
                .otherwise(
                    pl.col(index_name) - 1) \
                .alias(column_name))

In [ ]:
newdf.columns

In [ ]:
ret = pi.crossover(newdf, index_name, column_name)
ret.df.filter(pl.col(ret.column) == True)

In [ ]:
df.filter(pl.col('Symbol') == 'B').is_empty()

In [ ]:
lf = df.lazy()
lf

In [ ]:
same_symbol = 'same_symbol'
cross_up = 'cross_up'
cross_down = 'cross_down'
SYMBOL_COLUMN = 'Symbol'
column1 = 'Open'
column2 = 'Close'
lf = lf.with_columns((pl.col(SYMBOL_COLUMN).shift(1) == pl.col(SYMBOL_COLUMN)).alias(same_symbol))


In [ ]:
lf.collect()

In [ ]:

lf = lf.with_columns(
        (
            (pl.col(column1) > pl.col(column2))
            &
            (pl.col(column1).shift(1) < pl.col(column2).shift(1))
        ).alias(cross_up)
    )

In [ ]:
lf.collect()

In [ ]:

lf = lf.with_columns(
    (
        (pl.col(column1) < pl.col(column2))
        &
        (pl.col(column1).shift(1) > pl.col(column2).shift(1))
    ).alias(cross_down)
)

In [ ]:
lf.collect()

In [ ]:
lf.with_columns(((pl.col(cross_up) | pl.col(cross_down)) & pl.col(same_symbol)).alias(column_name)).collect()